# MNIST Example

In [ ]:
using Random
Random.seed!(1234);  # For experiment reproducibility. Can set this to any value. 

In [ ]:
using Flux, Statistics 
using Flux.Data.MNIST
using Flux: onehotbatch

In [ ]:
imgs = MNIST.images()
typeof(imgs)

Let's dissect that a bit ... 

In [ ]:
using Plots 

In [ ]:
Plots.scatter(imgs[1], size = (200, 200))  # Julia plots just 'knows' that this is suitable as an image

In [ ]:
@show size(imgs)     
@show size(imgs[1]) ;

## Loading and Encoding Labels 

In [ ]:
labels = MNIST.labels();

In [ ]:
labels[1:5]

In [ ]:
@show size(labels)

In [ ]:
labels = onehotbatch(MNIST.labels(), 0:9)    # convert to one-hot encoding 
labels[:, 1:5]  # show first five labels 

In [ ]:
@show size(labels);

## Creating Batches and Preparing Data 

We can use the parition function in Julia base library to create batches. This way, we can easily manage the memory and speed trade-off. 

In [ ]:
using Base.Iterators: partition

In [ ]:
for i in partition(1:10, 5)    # create equal partitions of size 2 
    println(i)
end

Let's reshape the "Gray Scale" 28x28 to a float32. That's going to take a bit of effort.

In [ ]:
imgs[1]

In [ ]:
typeof(imgs[1][1])   # this needs to be converted to float 

In [ ]:
typeof(imgs)

In [ ]:
typeof(float.(imgs))

In [ ]:
imgs = convert(Array{Array{   Float32    , 2}, 1}, float.(imgs));    # hack needed to convert everything to Float32

In [ ]:
size(imgs[1])

In [ ]:
typeof(imgs)

Let's make sure our model can handle more than one "channels" (for future proofing it.)

In [ ]:
t = (cat(imgs[1:2]..., dims = 4),   labels[:,1:2]);

In [ ]:
t[1]

In [ ]:
batch_size = 1000;

In [ ]:
train = [ 
          (cat(imgs[i]..., dims = 4),   labels[:,i])
            for i in partition(1:60_000, batch_size)
        ]
;

In [ ]:
@show size(train)
@show length(train[1])
@show size(train[1][1]);
@show size(train[1][2]);

## Prepare Test Data

In [ ]:
test_images = MNIST.images(:test)[1:1000];

In [ ]:
test_images = float.(test_images);

In [ ]:
typeof(test_images)

In [ ]:
test_images = convert(Array{Array{Float32, 2}, 1}, test_images);

In [ ]:
typeof(test_images)

In [ ]:
# we have just a single batch for testing 
tX = cat(test_images..., dims = 4);
tY = onehotbatch(MNIST.labels(:test)[1:1000], 0:9);

In [ ]:
@show size(tX)
@show size(tY);

## The Actual CNN Model 

In [ ]:
model = Chain(
  Conv((3,3), 1=>16, relu),
  MaxPool((2,2)),
  Conv((3,3), 16=>8, relu),
  MaxPool((2,2)),
  flatten,
  Dense(200, 10),
  softmax)

In [ ]:
using Flux: onecold, crossentropy, throttle

In [ ]:
m = model 

In [ ]:
opt = ADAM()
# opt = Descent() 
# opt = Momentum()
# opt = Nesterov()
# opt = NADAM() 

In [ ]:
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))      # onecold is the inverse operation to one-hot

In [ ]:
loss(x, y) = Flux.Losses.crossentropy(m(x), y)

In [ ]:
evalcb = throttle(  () -> @show(accuracy(tX, tY)),    10)

In [ ]:
Flux.train!(loss, params(m), train, opt, cb = evalcb)

## Let's See Some Clean Code

In [1]:
using Flux, Statistics 
using Flux.Data.MNIST
using Flux: onehotbatch
using Base.Iterators: partition
using Flux: onecold, crossentropy, throttle

function get_training_test_data(batch_size=1000) 
    # load and fix train data 
    train_labels = onehotbatch(MNIST.labels(), 0:9) 
    
    # label and imgs are combined for training 
    imgs = MNIST.images()
    imgs = convert(Array{Array{Float32, 2}, 1}, float.(imgs));    # hack needed to convert everything to Float32
    train = [ (cat(imgs[i]..., dims = 4),   train_labels[:,i])
               for i in partition(1:60_000, batch_size)  ]
    
    # test images and labels sent separately 
    test_imgs = MNIST.images(:test)[1:1000];
    test_imgs = convert(Array{Array{Float32, 2}, 1}, float.(test_imgs));
    test_imgs = cat(test_imgs..., dims = 4);
    test_labels = onehotbatch(MNIST.labels(:test)[1:1000], 0:9);
    
    return (train, test_imgs, test_labels)
    
end

function build_model()
    model = Chain(
        Conv((3,3), 1=>16, relu),
        MaxPool((2,2)),
        Conv((3,3), 16=>8, relu),
        MaxPool((2,2)),
        flatten,
        Dense(200, 10),
        softmax
    )
end 

function train_model!(; iterations=1, opt = ADAM())
    m = build_model() 
    
    train, tX, tY =  get_training_test_data()
    loss(x, y) = Flux.Losses.crossentropy(m(x), y)
    
    accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))     
    evalcb = throttle(() -> @show(accuracy(tX, tY)), 10)
    
    for i in 1:iterations
        Flux.train!(loss, params(m), train, opt, cb = evalcb)
    end 
end 

train_model! (generic function with 1 method)

In [2]:
train_model!(iterations=3, opt=ADAM())

accuracy(tX, tY) = 0.189
accuracy(tX, tY) = 0.657
accuracy(tX, tY) = 0.846
accuracy(tX, tY) = 0.887


In [3]:
train_model!(iterations=3, opt=NADAM())

accuracy(tX, tY) = 0.169
accuracy(tX, tY) = 0.677
accuracy(tX, tY) = 0.835
accuracy(tX, tY) = 0.894
accuracy(tX, tY) = 0.916


In [4]:
train_model!(iterations=3, opt=RADAM())

accuracy(tX, tY) = 0.108
accuracy(tX, tY) = 0.141
accuracy(tX, tY) = 0.182
accuracy(tX, tY) = 0.373
accuracy(tX, tY) = 0.661


In [5]:
train_model!(iterations=3, opt=Descent())

accuracy(tX, tY) = 0.126
accuracy(tX, tY) = 0.601
accuracy(tX, tY) = 0.88
accuracy(tX, tY) = 0.908
accuracy(tX, tY) = 0.918


In [6]:
train_model!(iterations=3, opt=Descent(0.4))

accuracy(tX, tY) = 0.118
accuracy(tX, tY) = 0.796
accuracy(tX, tY) = 0.937
accuracy(tX, tY) = 0.951
accuracy(tX, tY) = 0.959


More optimizers and options here: https://fluxml.ai/Flux.jl/stable/training/optimisers/